This is notebook to analyse sentiments using yiyanghkust/finbert-tone from hugging face. we will test the model using pipeline and without pipeline. Then train the model using twitter reivews data and then validate the model.

In [ ]:
#pip install datasets

In [ ]:
!pip install accelerate -U

In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, Trainer, TrainingArguments

from datasets import Dataset

In [ ]:
#check if gpu is available
if torch.cuda.is_available():
  print("Cuda availale. GPU will used for computation")
  device='cuda:0'
else:
  print("Cpu will e use")
  device='cpu'
torch.cuda.is_available()

Cpu will e use


False

In [ ]:
#define model path on hugging face and load copy of the model with pointing it to GPU.
model_name='yiyanghkust/finbert-tone'
model= AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Approch fo use the model and tokenizer without using pipeline. Its just consume it.

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer

BertTokenizerFast(name_or_path='yiyanghkust/finbert-tone', vocab_size=30873, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
#model configuration. It has encoders like Positvie to 1, Negative to 2 etc. and other confuration properties.
model.config

BertConfig {
  "_name_or_path": "yiyanghkust/finbert-tone",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Neutral",
    "1": "Positive",
    "2": "Negative"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 2,
    "Neutral": 0,
    "Positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30873
}

In [ ]:
#print label encoders
id_2_label=model.config.id2label
id_2_label

{0: 'Neutral', 1: 'Positive', 2: 'Negative'}

In [ ]:
#provide input sentence and tokenize it
sentence = 'The market outlook is very positive. thank to ne economic policies'
#pt is pytorch. padding is the padding that means if we have string 200 length but set max_length is 512 then it will pad with something.
inputs=tokenizer(sentence,return_tensors='pt',padding=True, truncation=True, max_length=512)
inputs

{'input_ids': tensor([[   3,    6,   52,  954,   17,  190,  483,   48,  332,    9, 5454,  289,
          693,    4]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
#move all tensors to gpu
inputs={k:v.to(device) for k,v in inputs.items()}
inputs

{'input_ids': tensor([[   3,    6,   52,  954,   17,  190,  483,   48,  332,    9, 5454,  289,
           693,    4]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
#use pytorch or tensorflow(in this case pytorch) to make prediciton on given inputs. it return prediction score for each argument
with torch.no_grad():
  outputs=model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-7.0768, 11.6335, -5.7233]]), hidden_states=None, attentions=None)

In [ ]:
#get max argument predicted score. it will return poition of maximum score
predictions=np.argmax(outputs.logits.cpu().numpy(),axis=1)
predictions

array([1])

In [ ]:
#decode predicted label
mapped_array=[id_2_label[element] for element in predictions]
print(mapped_array)

['Positive']


# **Pipline Approach**
The transformer has pipeline that will do auto - tokenize, perform prediciton, argmax  etc.

In [ ]:
model_name='yiyanghkust/finbert-tone'
sentiment_pipeline = pipeline("sentiment-analysis",model=model_name,batch_size=128,device=device)

In [ ]:
sentiment_pipeline(sentence)

[{'label': 'Positive', 'score': 1.0}]

In [ ]:
# @title using dataset
df = pd.read_csv('/content/drive/MyDrive/reviewsdata.txt',sep='/n',names=['text'])

<ipython-input-49-30dbd658cc48>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/drive/MyDrive/reviewsdata.txt',sep='/n',names=['text'])


In [ ]:
df.head()

,text
0,the happy bastard's quick movie review
1,damn that y2k bug .
2,it's got a head start in this movie starring j...
3,little do they know the power within . . .
4,going for the gore and bringing on a few actio...


In [ ]:
preds = sentiment_pipeline(df['text'].tolist())

In [ ]:
df['pred_sentiment'] = [pred['label'] for pred in preds]
df['pred_score'] = [pred['score'] for pred in preds]
df.to_csv('/content/drive/MyDrive/output1.csv')

In [ ]:
df.groupby('pred_sentiment').size()

pred_sentiment
Negative     8
Neutral     41
Positive     6
dtype: int64

Finetune model using trainer class from hugging face

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/reviewsdata/twitter_training.csv',names=['id','source','origital_sentiment','text']).dropna()
df_train.head()

,id,source,origital_sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
model.config.label2id

{'Positive': 1, 'Negative': 2, 'Neutral': 0}

In [ ]:
#read files and encode labels
df_train = pd.read_csv('/content/drive/MyDrive/reviewsdata/twitter_training.csv',names=['id','source','original_sentiment','text']).dropna()
df_train['label'] = df_train['original_sentiment'].apply(lambda l:int(model.config.label2id[l]))
df_val = pd.read_csv('/content/drive/MyDrive/reviewsdata/twitter_validation.csv',names=['id','source','original_sentiment','text']).dropna()
df_val['label'] = df_train['original_sentiment'].apply(lambda l:int(model.config.label2id[l]))
df_train.head()

,id,source,original_sentiment,text,label
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,1
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,1
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,1
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,1
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,1


In [ ]:
#convert pandas datasets to hugging dataset
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)

print(dataset_train)

Dataset({
    features: ['id', 'source', 'original_sentiment', 'text', 'label', '__index_level_0__'],
    num_rows: 73996
})


In [ ]:
#tokenize the dataset. That means extract words
dataset_train = dataset_train.map(lambda e:tokenizer(e['text'],truncation=True,padding='max_length',max_length=128),batched=True)
dataset_val = dataset_val.map(lambda e:tokenizer(e['text'],truncation=True,padding='max_length',max_length=128),batched=True)

Map:   0%|          | 0/73996 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#setting the dataset format for pytorch
dataset_train.set_format(type='torch',columns=['input_ids','token_type_ids','attention_mask','label'],dtype=torch.long)
#dataset_train['label'] = dataset_train['label'].to(torch.int64)
dataset_val.set_format(type='torch',columns=['input_ids','token_type_ids','attention_mask','label'],dtype=torch.long)
#dataset_val['label'] =dataset_val['label'].to(torch.int64)

In [ ]:
#shuffle training dataset
dataset_train_shuffled = dataset_train.shuffle(seed=42)

In [ ]:
#Define trainer to find tune model
def compute_metrics(eval_pred):
  predictions , labels = eval_pred
  predictions = np.argmax(predictions,axis=1)
  return {'balanced_accuracy' : balanced_accuracy_score(predictions,labels),'accuracy':accuracy_score(predictions,labels)}

args = TrainingArguments(
    output_dir = 'temp/',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='steps', #log every x steps
    logging_steps=50, #log every 50 steps
    learning_rate=2e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.1,
    load_best_model_at_end=True,
    metric_for_best_model='balanced_accuracy'
)

trainer = Trainer(
    model=model,  #the instanticated transformer model to be trained
    args = args,    #trainig arguments, defined above
    train_dataset=dataset_train_shuffled,  #traing dataset
    eval_dataset=dataset_val,        #evaluation dataset
    compute_metrics=compute_metrics

)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
#train the model
trainer.train()
#x = torch.tensor(train).to(torch.int64)

In [ ]:
#predict for validate dataset
predictions = trainer.predict(dataset_val)
predictions

In [ ]:
#save retrained model to local path
model_path = '/content/drive/MyDrive/reviewsdata/sentiment_model'
trainer.model.save_pretrained(model_path)

#save tokenizer associated with the model
tokenizer.save_pretrained(mode_path)

In [ ]:
#load the trained model into pipelines
trained_pipeline = pipeline("text-classification",model=model_path,tokenizer=model_path,device=device)

In [ ]:
#make predictions and evaluate accuracy
preds=trained_pipeline(df_test['text'].tolist())
df_test['prediction']=[pred['label'] for pred in preds]

In [ ]:
#calculate  the balance accuracy score
score = balanced_accuracy_score(df_test['Sentiment_Label_Text'],df_test['prediction'])
print(f'Balanced Accuracy Score:  {score}')

In [ ]:
#calculate  the accuracy score
score = accuracy_score(df_test['Sentiment_Label_Text'],df_test['prediction'])
print(f'Accuracy Score:  {score}')